In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from auctioneer import *
from train_crypto_bot import *
pd.options.mode.chained_assignment = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
STARTING_BALANCE = 100_000

In [4]:
s3 = create_s3()
data = load_data(s3, None, None)
model = encoder_model(HISTORY_STEPS, TARGET_STEPS, 3)
model.load_weights('../../models/checkpoint.ckpt')
scalers = joblib.load('../../models/scalers.gz')

14:01:44: Reading data
2022-12-08 14:01:49.525301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def one_day(date, portfolio, data, model, scalers):
    data = data.loc[:date]
    predictions = predict_forward(data, model, scalers, HISTORY_STEPS)
    terminal_prices = {data.columns[i]: predictions[-1,i] for i in range(data.shape[1])}

    orders = {}
    for symbol in terminal_prices.keys():
        current_price = data[symbol].values[-1]
        predicted_price = terminal_prices[symbol]

        if predicted_price > current_price:
            amount = 1
            orders[symbol] = {'action': 'buy', 'amount': amount}
        elif predicted_price < current_price:
            amount = portfolio.holdings.get(symbol, 0)
            amount = max([1, amount // 10])
            orders[symbol] = {'action': 'sell', 'amount': amount}
    
    return orders

In [22]:
def backtest(data, model, scalers):
    portfolio = Portfolio(STARTING_BALANCE, data)


    for i, date in enumerate(data.index):
        if i < HISTORY_STEPS:
            continue    
        orders = one_day(date, portfolio, data, model, scalers)
        if orders is not None:
            portfolio.execute(date, orders)

        if (i % 25) == 0:
            print(i)
            print(orders)
    return portfolio

def windowed_backtest(data,model, scalers, window_size, stride = 1):
    logger.info('Windowed test')
    results = []
    for i in range(0, data.shape[0]-window_size, stride):
        tmp = data.iloc[i:i+window_size]
        result = backtest(tmp,model, scalers)
        log = result.transaction_log_summary()

        long_result = long_return(tmp, STARTING_BALANCE)
        results.append({
            'StartDate': data.index[i], 
            'EndValue': result.value()['Total'],
            'LongValue': long_result,
            'N_Buys': log['buys'][0],
            'N_Sells': log['sells'][0],
        })
    
    return pd.DataFrame(results)

In [23]:
start_index = data.index.get_loc(data.loc[END_DATE].index[0]) - HISTORY_STEPS

In [24]:
eval_data = data.iloc[start_index:]
eval_data = eval_data.head(1000).bfill().ffill()

In [21]:
portfolio = Portfolio(STARTING_BALANCE, eval_data)
one_day(eval_data.index[400], portfolio, eval_data, model, scalers)

{'BTC/USD': {'action': 'buy', 'amount': 1},
 'ETH/USD': {'action': 'buy', 'amount': 1},
 'SOL/USD': {'action': 'buy', 'amount': 1}}

In [26]:
x = backtest(eval_data, model, scalers)

250
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'action': 'buy', 'amount': 1}, 'SOL/USD': {'action': 'buy', 'amount': 1}}
275
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'action': 'buy', 'amount': 1}, 'SOL/USD': {'action': 'buy', 'amount': 1}}
300
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'action': 'buy', 'amount': 1}, 'SOL/USD': {'action': 'buy', 'amount': 1}}
325
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'action': 'buy', 'amount': 1}, 'SOL/USD': {'action': 'buy', 'amount': 1}}
350
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'action': 'buy', 'amount': 1}, 'SOL/USD': {'action': 'buy', 'amount': 1}}
375
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'action': 'buy', 'amount': 1}, 'SOL/USD': {'action': 'buy', 'amount': 1}}
400
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'action': 'buy', 'amount': 1}, 'SOL/USD': {'action': 'buy', 'amount': 1}}
425
{'BTC/USD': {'action': 'buy', 'amount': 1}, 'ETH/USD': {'a